**In this project, a face recognition and analysis of the results will be performed using the following procedures:**
1. face detection
2. facial features extraction
3. classification
4. analysis of the results

**The total Project mark is 20. The lab report shall include:**
- Description of the implemented project (introduction, procedure and analysis, conclusion) (6 marks).
- The project flow block-diagram, illustrations, graphs (such as DET, ROC) and their analysis (6 marks).
- Code or any modifications to the code (8 marks).

In [77]:
#%pip install opencv-python

In [ ]:
import cv2
import skimage
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from sklearn.svm import SVC
# from sklearn.svm import SVM

In [ ]:
#loads images from folder and returns imges and labels
#folder - path to folder with images
def get_data(folder):
    images = []
    images_1D = []
    labels = []
    
    # Iterate over each subfolder in the main directory
    for subfolder in os.listdir(folder):
        if os.path.isdir(os.path.join(folder, subfolder)): 
            # Iterate over each image in the subfolder
            for filename in os.listdir(os.path.join(folder, subfolder)):
                path = os.path.join(folder, subfolder, filename)
                if os.path.isfile(path):
                    #read image
                    img = cv2.imread(path, 0)
                    #get label
                    label = filename[filename.rfind('s') + 1:filename.rfind('\\')]
                    label = label.split('.')[0]
                    if img is not None:
                        #base images stored in array
                        # for ploting base images
                        images.append(img)

                        #get 1D image and store to array
                        img = cv2.resize(img, (1, (len(img) * len(img[0]))))
                        img = np.float32(np.array(img) / 255.0)
                        images_1D.append(img)

                        #add label to array
                        labels.append(int(label))
    
    return images, np.array(images_1D)[:,:,0], np.array(labels)  # Return images and Labels

def get_lbph(images):
    lbp_images = []
    for image in images:
        lbp = local_binary_pattern(image, 12, 3)
        lbp_images.append(lbp)
    return lbp_images

def train_svm(train_path, test_path, C=5.0, gamma=0.001):
    train_data, train_label = get_data(train_path)
    test_data, test_label = get_data(test_path)
    
    svm = SVC(C=C, gamma=gamma, probability=True)
    svm.fit(train_data, train_label)
    
    probability_matrix = svm.predict_proba(test_data)
    prediction = np.argmax(probability_matrix, 1)
    result = prediction == test_label
    accuracy = np.sum(result) / len(result)
    
    print("Accuracy:", accuracy)

def plot_images(images, labels, rows, cols):
    fig, axes = plt.subplots(rows, cols, figsize=(10, 5))
    for i, image in enumerate(images):
        axes[i].imshow(image, cmap='gray')  # Specify cmap='gray' to display grayscale images
        axes[i].axis('off')
    plt.show()

images, images_1D, labels = get_data('./ATT_dataset')

lbp = get_lbph(images)

fig = None
axes = None

if len(images) > 0:
    fig, axes = plt.subplots(1, min(len(images), 6), figsize=(10, 5))
    for i, image in enumerate(images[:6]):
        axes[i].imshow(image, cmap='gray')  # Specify cmap='gray' to display grayscale images
        axes[i].axis('off')
    plt.show()
else:
    print("No images found.")

lbp = get_lbph(images)
fig, axes = plt.subplots(1, min(len(lbp), 6), figsize=(10, 5))
for i, lbp_image in enumerate(lbp[:6]):
    axes[i].imshow(lbp_image, cmap='gray')
    axes[i].axis('off')
plt.show()



##### This is part 3 of the provided instructions

You can implement Face detection in Python language using OpenCV library. The following code load the face image and convert it to gray scale:

In [ ]:
#%ls dog_with_headphones.jpg

In [ ]:
iface_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
image_path = './dog_with_headphones.jpg'

if os.path.exists(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        print(f"Image file '{image_path}' is empty.")
else:
    print(f"Image file '{image_path}' does not exist.")


Next the Haar-Cascades is used for the face detection:

In [ ]:
face_box = face_cascade.detectMultiScale(img_gray, 1.1, 4)

Here, 1.1 is a scale factor, and 4 is the number of neighbours to retain for the rectangular around the face.
An alternative approach to the frontal face cascade approach is LPB-based Haar cascades: https://github.com/opencv/opencv/tree/master/data/lbpcascades 

Two good full-run examples of face detection can be found here: https://www.datacamp.com/community/tutorials/face-detection-python-opencv#face-detection

and here: https://scikit-image.org/docs/dev/auto_examples/applications/plot_face_detection.html

For a lot of the project, you may want to create your own custom load function. Here is an example:

In [ ]:
import cv2
import numpy as np
from glob import glob
# custom load data function to load images and record subject labels
def get_data(path):
    paths = glob(path, recursive=True)
    data = [] #list of images
    label = [] #list of labels
    for path in paths :
        img = cv2.imread(path,0) # read image
        subject_label = path[path.rfind('s')+1: path.rfind ('\\')] # extract the label
        # pre=processing step
        # can resize , rescale , normalize
        img = cv2.resize(img, (1, len(img)*len(img[0]))) # reshape image to a 1D vector
        img = np.float32(np.array(img)/255.0) # normalize to 0=1 value
        # can apply LBP , PCA or other forms of feature extraction
        # append images and labels
        data.append(img)
        # decrease all labels by 1 since subject labels start from 1
        label.append(int(subject_label)-1)
        
    # return np.array(data)[:,:,0], np.array(label)
    return np.array(data), np.array(label)

For classification (face recognition) with SVM, you can still use the version implemented in OpenCV; however, you will not get the probability values required to do the analysis required. Check the following link for the documentation and sample code:

https://docs.opencv.org/4.x/d0/dcc/tutorial_non_linear_svms.html

In [ ]:
#ORL dataset contains 40 subjects with 10 images per subject
#5 images are used for training and the remaining 5 images are used for testing
train_path = r'ORL\\Train Data\\*\\*'
test_path = r'ORL\\Test Data\\*\\*'
train_data, train_label = get_data(train_path)
test_data, test_label = get_data(test_path)
from sklearn.neural_network import MLPClassifier
import numpy as np
train_path = r'ORL\\Train Data\\*\\*'
test_path = r'ORL\\Test Data\\*\\*'
# customize get data function to include pre=processing methods ( adding PCA or LBP)

train_data , train_label = get_data(train_path) # use the previous custom get data function
test_data , test_label = get_data(test_path) # use the previous custom get data function
# create MLP with 3 layers of perceptrons
# first layers has 128 neurons then 64 then another 128
# experiment with different layers / neurons
# experiment with different learning rate
mlp = MLPClassifier(hidden_layer_sizes =(128 ,64 ,128), learning_rate_init = 0.001, random_state =1)
mlp.fit(train_data , train_label)
# probability matrix NxM where N is number of samples and M is the number of classes
probability_matrix = mlp.predict_proba(test_data)
# calculate accuracy
prediction = np.argmax(probability_matrix,1)
result = prediction == test_label
accuracy = np.sum(result)/len(result)

##### This is part 4.1.3 in their provided example
Both classifiers, SVM and MLP, implementation in the Scikit-Learn has similar interface, where we call .fit(...) for training (using the training set) and .predict(...) to test (using the test set only). In the example, we used .predict_proba(...) to get the probability matrix. The matrix can be used to calculate additional metrics and graphs such as the ROC (FPR vs. TPR) or DET (FPR vs. FNR) curves

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve , det_curve
# false positive rate (FPR)
# true positive rate (TPR)
# false negative rate (FNR)
# 40 subjects by 100 values
roc_fpr = np.zeros((40 ,100))
roc_tpr = np.zeros((40 ,100))
det_fpr = np.zeros((40 ,100))
det_fnr = np.zeros((40 ,100))
fig,[ax_roc, ax_det] = plt.subplots(1, 2, figsize=(11, 5))
# iterate through number of subjects (40)
# calculating the ROC and DET curve for each subject
for i in range (40):
    # find the fpr by tpr values for ROC
    fpr, tpr, = roc_curve(test_label[:]==i, probability_matrix[:,i])
    # interpolate rates so each curve contains 100 values
    roc_fpr [i] = np.linspace (min(fpr),max(fpr),100)    
    roc_tpr [i] = np.interp (roc_fpr[i],fpr,tpr)
    # find the fpr by fnr values for DET
    fpr, fnr, = det_curve(test_label [:]==i, probability_matrix[:,i])
    # interpolate rates so each curve contains 100 values
    det_fpr [i] = np.linspace(min(fpr),max(fpr),100)
    det_fnr [i] = np.interp(det_fpr[i],fpr ,fnr)
# average each subject ’s ROC curve to get the average ROC curve of system
roc_mid_fpr = np.mean(roc_fpr,0)
roc_mid_tpr = np.mean(roc_tpr,0)
ax_roc.plot(roc_mid_fpr, roc_mid_tpr)
# average each subject ’s DET curve to get the average DET curve of system
det_mid_fpr = np.mean(det_fpr ,0)
det_mid_fnr = np.mean(det_fnr ,0)
ax_det.plot(roc_mid_fpr, roc_mid_tpr)
ax_roc.set_xlabel('FPR')
ax_roc.set_ylabel('TPR')
ax_roc.set_title(" Receiver Operating Characteristic (ROC) curves ")
ax_det.set_xlabel('FPR')
ax_det.set_ylabel('FNR')
ax_det.set_title(" Detection Error Tradeoff (DET) curves ")
plt.show()